In [15]:
from lark import Lark
from lark.visitors import Transformer
from LabeledTransitionSystem import *

In [16]:
aut_parser = Lark.open("grammars/aut.lark", parser="lalr")
mcf_parser = Lark.open("grammars/mcf.lark", parser="lalr")

In [17]:
system = aut_parser.parse(open("../material/dining/dining_10.aut").read())
formula = mcf_parser.parse(open("../material/dining/invariantly_inevitably_eat.mcf").read())

In [21]:
class LTSTransformer(Transformer):
    def string(self, s):
        return s[1:-1]
    
    number = int

    FIRST_STATE = number
    NR_OF_TRANSITIONS = number
    NR_OF_STATES = number

    def aut_header(self, header):
        init_state, _, n_states = header
        return [set(range(n_states)), {init_state}]

    START_STATE = number
    LABEL = string
    END_STATE = number

    def aut_edge(self, transition):
        return tuple(transition)

    # args is a list containing (in this order)
    # set of states, initial_states and then transitions.
    # Order is always given, since all files start with aut_header
    # and then go on to specify transitions.
    def start(self, args):
        args = args[0] + args[1:]
        lts = LabeledTransitionSystem(*args)
        return lts


lts = LTSTransformer().transform(system)
%store lts

Stored 'lts' (LabeledTransitionSystem)
